In [ ]:
import time
import tonic
import importlib
import numpy as np
import tonic.transforms as transforms
import tensorflow as tf

from source.utilities import *
from source.clusterGAN import clusterGAN

# author code includes deprecated packages so let's suppress the deprecation warnings
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
if type(tf.contrib) != type(tf): tf.contrib._warning = None

### parameters

In [ ]:
# dataset parameters
dataset_name       = 'DVSGESTURE'
frame_time         = 1000
subsample          = 100 # take a sample of the dataset

# clusterGAN parameters
batch_size         = 64
latent_dim         = 30
beta_cycle_gen     = 10.0
beta_cycle_label   = 10.0
train              = True
timestamp          = ''

### load modules

In [ ]:
data = importlib.import_module('source.' + dataset_name.lower())
model = importlib.import_module('source.' + dataset_name.lower() + '.clus_wgan')

### building model and training

In [ ]:
data_sampler = data.DataSampler(frame_time, subsample)
num_classes = data_sampler.num_classes
z_dim = latent_dim + num_classes
discriminator = model.Discriminator()
generator = model.Generator(z_dim=z_dim)
encoder = model.Encoder(z_dim=z_dim, latent_dim = latent_dim)

# initialising clusterGAN
cluster_gan = clusterGAN(generator, 
                         discriminator, 
                         encoder, 
                         dataset_name.lower(),
                         data_sampler, 
                         num_classes, 
                         latent_dim, 
                         batch_size, 
                         beta_cycle_gen, 
                         beta_cycle_label)

# training
if train:
    cluster_gan.train()
else:
    cluster_gan.load(pre_trained=False, timestamp = timestamp)
    cluster_gan.recon_enc(timestamp)